In [3]:
from ortools.linear_solver import pywraplp
import math
import numpy as np
import pandas as pd
import os

#os.getcwd()

# df3: merged +40.xlsx with BPC Assy-Coat-Mold .xlsx
# Load spreadsheet
plus_40v3_path = os.path.abspath("plus_40 v3.ipynb")

plus_40v3 = os.path.join(os.path.dirname(plus_40v3_path), "data\Cycle times per recipe and location.xlsx")
df1 = pd.read_excel(plus_40v3,sheet_name="+40",header=None)

def main():
    # Data

    time = [
        [204, 240, 1920],# 5A8105
        [204, 240, 1920],
        [282, 282, 540],#588100
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],# 5A8105
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [277, 258, 1200],# 9
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],# 12
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],# 15
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 217, 1200],# 18
        [277, 217, 1200],
        [277, 217, 1200],
        [277, 217, 1200],# 21
        [277, 217, 1200],
        [203, 230, 1200],#23
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200]
    ]


    num_tasks = len(time[0])
    num_parts = len(time_0)

    num_racks = 0
    for i in range(num_parts):
        num_racks += math.ceil(time_0[i][3] / time_0[i][4])

    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver('SolveSimpleSystem', pywraplp.Solver.SAT_INTEGER_PROGRAMMING)


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if rack i is assigned to task j.
    x = {}
    for i in range(num_racks):
        for j in range(num_racks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # Each rack is assigned to at most 1 task.
    for i in range(num_racks):
        solver.Add(solver.Sum([x[i, j] for j in range(num_racks)]) <= 1)

    # Each task is assigned to exactly one rack.
    for j in range(num_racks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_racks)]) == 1)

    # 
    wt_p = [] # wt_p
    wt_tp = [] # wt_tp
    for i in range(num_racks):
        wt_p.append(solver.Sum([x[i, j] * time[j][0] \
                                for j in range(num_racks)]))
        wt_tp.append(solver.Sum([x[i, j] * time[j][1] \
                                 for j in range(num_racks)]))

    time[-1][0] = 0
    time[-1][1] = 0
    time[-2][1] = 0
    for i in np.arange(0,19):
        wt_p[i+1] = wt_p[i] + max(time[i-1][0],time[i][0],time[i+1][0])

    for i in np.arange(19,num_racks-1):
        wt_p[i+1] = wt_p[i] + max(time[i-1][0],time[i][0],time[i][0],\
                                  time[i-21][1],time[i-20][1],time[i-19][1])

    for i in np.arange(0,num_racks-21):
        wt_p[i+21] = wt_tp[i]

    # Objective        
    solver.Minimize(wt_tp[num_racks-1])

    # Solve
    status = solver.Solve()

        # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        #print(f'Total time = {wt_tp[num_racks-1].solution_value()}\n')
        for i in range(num_racks):
            for j in range(num_racks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print('x[',i,',', j,']=',x[i,j].solution_value())
                    print(f'rack {i} assigned to position {j}.')
                else:
                    x[i,j] = x[i, j].solution_value()
    else:
        print('No solution found.')
        
if __name__ == '__main__':
    main()

x[ 0 , 4 ]= 1.0
rack 0 assigned to position 4.
x[ 1 , 1 ]= 1.0
rack 1 assigned to position 1.
x[ 2 , 2 ]= 1.0
rack 2 assigned to position 2.
x[ 3 , 3 ]= 1.0
rack 3 assigned to position 3.
x[ 4 , 6 ]= 1.0
rack 4 assigned to position 6.
x[ 5 , 5 ]= 1.0
rack 5 assigned to position 5.
x[ 6 , 14 ]= 1.0
rack 6 assigned to position 14.
x[ 7 , 7 ]= 1.0
rack 7 assigned to position 7.
x[ 8 , 8 ]= 1.0
rack 8 assigned to position 8.
x[ 9 , 9 ]= 1.0
rack 9 assigned to position 9.
x[ 10 , 10 ]= 1.0
rack 10 assigned to position 10.
x[ 11 , 11 ]= 1.0
rack 11 assigned to position 11.
x[ 12 , 12 ]= 1.0
rack 12 assigned to position 12.
x[ 13 , 13 ]= 1.0
rack 13 assigned to position 13.
x[ 14 , 15 ]= 1.0
rack 14 assigned to position 15.
x[ 15 , 17 ]= 1.0
rack 15 assigned to position 17.
x[ 16 , 16 ]= 1.0
rack 16 assigned to position 16.
x[ 17 , 19 ]= 1.0
rack 17 assigned to position 19.
x[ 18 , 26 ]= 1.0
rack 18 assigned to position 26.
x[ 19 , 20 ]= 1.0
rack 19 assigned to position 20.
x[ 20 , 22 ]= 1.0


In [ ]:
from ortools.linear_solver import pywraplp
import math
import numpy as np

def main():
    # Data
    time_0 = [
    [282, 282, 540, 341, 38],
    [277, 258, 1200, 696, 84],
    [277, 217, 1200, 420, 84],
    [203, 230, 1200, 588, 84],
    [210, 266, 1200, 588, 84]
    ]

    time = [
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [282, 282, 540],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 258, 1200],
        [277, 217, 1200],
        [277, 217, 1200],
        [277, 217, 1200],
        [277, 217, 1200],
        [277, 217, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [203, 230, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200],
        [210, 266, 1200]
    ]


    num_tasks = len(time[0])
    num_parts = len(time_0)

    num_racks = 0
    for i in range(num_parts):
        num_racks += math.ceil(time_0[i][3] / time_0[i][4])

    # Solver
    # Create the mip solver with the SCIP backend.
    solver = pywraplp.Solver('SolveSimpleSystem', pywraplp.Solver.SAT_INTEGER_PROGRAMMING)


    # Variables
    # x[i, j] is an array of 0-1 variables, which will be 1
    # if rack i is assigned to task j.
    x = {}
    for i in range(num_racks):
        for j in range(num_racks):
            x[i, j] = solver.IntVar(0, 1, '')

    # Constraints
    # 1. Each rack is assigned to at most 1 task.
    for i in range(num_racks):
        solver.Add(solver.Sum([x[i, j] for j in range(num_racks)]) <= 1)

    # 2. Each task is assigned to exactly one rack.
    for j in range(num_racks):
        solver.Add(solver.Sum([x[i, j] for i in range(num_racks)]) == 1)

    # constraint 3,4,5
    wt_p = [] 
    wt_tp = [] 
    wb_p = [0]
    wb_tp = []
    for i in range(num_racks):
        wt_p.append(solver.Sum([x[i, j] * time[j][0] \
                                for j in range(num_racks)]))
        wt_tp.append(solver.Sum([x[i, j] * time[j][1] \
                                 for j in range(num_racks)]))
        wb_p.append(wb_p[i] + wt_p[i])
        
    for i in range(num_racks):
        #wb_tp[i] =         
        wb_tp.append(wb_p[i+20] if i < num_racks - 20 else wb_tp[i-1] + wt_tp[i-1]
)
        wt_p[i] + wb_p[i] <= wb_tp[i]
        
    time[-1][0] = 0
    time[-1][1] = 0
    time[-2][1] = 0
    
    # 6. the completion time for the (i+1)th rack in prime tank
    for i in np.arange(0,19):
        wt_p[i+1] + wb_p[i+1] == wt_p[i] + wb_p[i] + max(time[i-1][0],time[i][0],time[i+1][0])

    for i in np.arange(19,num_racks-1):
        wt_p[i+1] + wb_p[i+1] == wt_p[i] + wb_p[i] + max(time[i-1][0],time[i][0],time[i][0],\
                                  time[i-21][1],time[i-20][1],time[i-19][1])

    # 7. the completion time for the (i+21)th rack in prime tank
    # equals the completion time for the ith rack in Top Coat tank
    for i in np.arange(0,num_racks-21):
        wb_p[i+21] + wt_p[i+21] == wb_tp[i] + wt_tp[i]

    # Objective        
    solver.Minimize(wb_tp[num_racks-1] + wt_tp[num_racks-1])

    # Solve
    status = solver.Solve()

        # Print solution.
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        #print(f'Total time = {wt_tp[num_racks-1].solution_value()}\n')
        for i in range(num_racks):
            for j in range(num_racks):
                # Test if x[i,j] is 1 (with tolerance for floating point arithmetic).
                if x[i, j].solution_value() > 0.5:
                    print('x[',i,',', j,']=',x[i,j].solution_value())
                    print(f'rack {i} assigned to position {j}.')
                else:
                    x[i,j] = x[i, j].solution_value()
    else:
        print('No solution found.')
        
if __name__ == '__main__':
    main()